In [ ]:
from google.colab import drive
from google.colab import files
import pandas as pd

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv("drive/My Drive/Diploma/second_try/Code/other_language/events_german_2022.csv")

def transform_date(date):
  components = date.split('_')
  month = {"Januar": "01", "Februar": "02", "März": "03", 
           "April": "04", "Mai": "05", "Juni": "06",
           "Juli": "07", "August": "08", "September": "09", 
           "Oktober": "10", "November": "11", "Dezember": "12"}
  components[1] = components[1].split('.')[0]
  if len(components[1])==1:
    components[1] = '0' + components[1]
  return '2022' + '-' + month[components[2]]+ '-' + components[1]

df['text'] = df['title']
df['title'] = df['link'].str.split('/', expand=True)[2]
df['date'] = df['date'].apply(lambda x: transform_date(x))
df.head(2)

,Unnamed: 0,date,link,title,text
0,0,2022-01-01,/wiki/Berlin,Berlin,Berlin
1,1,2022-01-01,/wiki/Deutschland,Deutschland,Deutschland


# add views features

In [ ]:
!pip install pageviewapi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install -q swifter

In [1]:
from numpy import around
from collections.abc import Mapping
import pageviewapi
from datetime import datetime, timedelta
import swifter
import pandas as pd

In [2]:

# https://github.com/Comhttps://en.wikipedia.org/w/api.php?action=query&format=json&prop=revisions&titles=2021%E2%80%932022_Boulder_County_fires&rvprop=ids&rvlimit=1&rvstart=2022-01-01T00%3A00%3A00.010Zmonists/pageview-api
def get_n_views(title, from_, to_):
  try:
    res = sum([i['views'] for i in pageviewapi.per_article('en.wikipedia', title, from_, to_,
                        access='all-access', agent='all-agents', granularity='daily')['items']])
  except:
    #print(title, from_, to_)
    res = -1
  return res
  
def get_views_features(title, day):
  before_2 = datetime.strftime(datetime.strptime(day, '%Y%m%d') - timedelta(days=2), '%Y%m%d')
  after_2 = datetime.strftime(datetime.strptime(day, '%Y%m%d') + timedelta(days=2), '%Y%m%d')
  before_6 = datetime.strftime(datetime.strptime(day, '%Y%m%d') - timedelta(days=6), '%Y%m%d')

  around = get_n_views(title, before_2, after_2)
  before = get_n_views(title, before_6, before_2)

  boost = around/before

  prev_day = datetime.strftime(datetime.strptime(day, '%Y%m%d') - timedelta(days=1), '%Y%m%d')
  week_before = datetime.strftime(datetime.strptime(day, '%Y%m%d') - timedelta(days=7), '%Y%m%d')
  month_before = datetime.strftime(datetime.strptime(day, '%Y%m%d') - timedelta(days=30), '%Y%m%d')
  year_before = datetime.strftime(datetime.strptime(day, '%Y%m%d') - timedelta(days=365), '%Y%m%d')

  day_after = datetime.strftime(datetime.strptime(day, '%Y%m%d') + timedelta(days=1), '%Y%m%d')
  # week_after = datetime.strftime(datetime.strptime(day, '%Y%m%d') + timedelta(days=7), '%Y%m%d')
  # month_after = datetime.strftime(datetime.strptime(day, '%Y%m%d') + timedelta(days=30), '%Y%m%d')

  same_day_views = get_n_views(title, day, day)
  prev_day_views = get_n_views(title, prev_day, day)
  week_before_views = get_n_views(title, week_before, day)
  month_before_views = get_n_views(title, month_before, day)
  year_before_views = get_n_views(title, year_before, day)

  day_after_views = get_n_views(title, day, day_after)
  # week_after_views = get_n_views(title, day, week_after)
  # month_after_views = get_n_views(title, day, month_after)

  return [title, day,  around, before, boost, same_day_views, prev_day_views, week_before_views, month_before_views, 
          year_before_views, day_after_views#, week_after_views, month_after_views
          ]


In [ ]:
df['date'] = df['date'].astype(str)

In [ ]:
len(df)/2

8448.5

In [ ]:
views_features = df[:8448].swifter.apply(lambda x: get_views_features(x['title'], x['date']), axis=1)

In [ ]:
views_features

In [ ]:
views_features_df = pd.DataFrame(views_features.tolist(), columns=["title", "day", "around", "before", "boost", "same_day_views", "prev_day_views", "week_before_views", "month_before_views", 
          "year_before_views", "day_after_views"])

In [ ]:
views_features_df.to_csv('german_rev_views_2022_1.csv') 
files.download('german_rev_views_2022_1.csv')

In [ ]:
views_features = df[10000:].swifter.apply(lambda x: get_views_features(x['title'], x['date']), axis=1)

In [ ]:
views_features_df = pd.DataFrame(views_features.tolist(), columns=["title", "day", "around", "before", "boost", "same_day_views", "prev_day_views", "week_before_views", "month_before_views", 
          "year_before_views", "day_after_views"])
views_features_df.to_csv('german_rev_views_2022_2.csv') 
files.download('german_rev_views_2022_2.csv')

# Add topic features

In [ ]:
import json
import swifter
import requests

def get_topic(title):
  try:
    url = f'https://wikipedia-topic.wmcloud.org/api/v1/topic?lang=en&title={title}'
    r = requests.get(url).text
    r = json.loads(r)['results'][0] # score, topic
    a, b = r['score'], r['topic']
  except:
    a, b = -1, -1
  return title, a, b

a = "Prime_Minister_of_Haiti"
get_topic(a)

('Prime_Minister_of_Haiti',
 0.8634016513824463,
 'Geography.Regions.Americas.Central_America')

In [ ]:
len(set(df['title']))

1724

In [ ]:
df_cat = pd.DataFrame(set(df['title']), columns=['title'])

In [ ]:
category = df_cat.swifter.apply(lambda x: get_topic(x['title']), axis=1) 

Pandas Apply:   0%|          | 0/1724 [00:00<?, ?it/s]

In [ ]:

category.to_csv(f'german_events_category.csv')
files.download(f'german_events_category.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Add revision features

In [ ]:

import swifter
import requests
def flatten_data(y):
    out = {}

    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], a)
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, str(i) )
                i += 1
        else:
            out[name[:-1]] = x

    flatten(y)
    return out

def get_revision_by_date(title, date):
  try:
    query = f"https://en.wikipedia.org/w/api.php?action=query&format=json&prop=revisions&titles={title}&rvprop=ids&rvlimit=1&rvstart={date}T00%3A00%3A00.010Z"
    res = flatten_data(requests.get(query).json())['parenti']
  except:
    res = -1
  return res


def get_n_revisions(title, revision_from, revision_to):
  if revision_from==revision_to:
    return 0
  try:
    query = f"https://en.wikipedia.org/w/rest.php/v1/page/{title}/history/counts/edits?from={revision_from}&to={revision_to}"
    res = requests.get(query).json()
    # print(res)
    res = res['count']
    res +=1
  except:
    #print(title, from_, to_)
    res = -1
  return res

def substract_days(date, days):
  return datetime.strftime(datetime.strptime(date, '%Y-%m-%d') - timedelta(days=days), '%Y-%m-%d')

import datetime
from datetime import datetime, timedelta
def get_revision_features(title, date):
  # date = day[:4]+'-'+day[4:6]+'-'+day[6:8]
  same_day = get_revision_by_date(title, date)
  next_day =  get_revision_by_date(title, substract_days(date, -1))
  next_day_2 =  get_revision_by_date(title, substract_days(date, -2))

  prev_day = get_revision_by_date(title, substract_days(date, 1))
  before_2_days = get_revision_by_date(title, substract_days(date, 2))
  week_before =  get_revision_by_date(title, substract_days(date, 7))
  month_before =  get_revision_by_date(title, substract_days(date, 30))
  year_before = get_revision_by_date(title, substract_days(date, 365))
  week_after =  get_revision_by_date(title, substract_days(date, -7))

  # print(same_day, prev_day)
  same_day_views = get_n_revisions(title, prev_day, same_day)
  prev_day_views = get_n_revisions(title, before_2_days, prev_day)
  week_before_views = get_n_revisions(title, week_before, prev_day)
  month_before_views = get_n_revisions(title, month_before, prev_day)
  year_before_views = get_n_revisions(title, year_before, prev_day)
  week_after_views = get_n_revisions(title, prev_day, week_after)
  day_after_revs = get_n_revisions(date, same_day, next_day)
  day_after_2_revs = get_n_revisions(title, next_day, next_day_2)

  return [title, date, same_day_views, prev_day_views, week_before_views, month_before_views, 
          year_before_views, week_after_views, day_after_revs, day_after_2_revs]


In [ ]:
revisions = df[:1678].swifter.apply(lambda x: get_revision_features(x['title'], x['date']), axis=1)

Pandas Apply:   0%|          | 0/1678 [00:00<?, ?it/s]

In [ ]:

revisions_df = pd.DataFrame(revisions.tolist(), columns=["title", 'date', "same_day_views", "prev_day_views", "week_before_views", "month_before_views", 
          "year_before_views", "week_after_views", "day_after_revs", "day_after_2_revs"])

In [ ]:
revisions_df.to_csv('german_events_revisions_2022_1.csv') 
files.download('german_events_revisions_2022_1.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
revisions = df[1678:].swifter.apply(lambda x: get_revision_features(x['title'], x['date']), axis=1)

Pandas Apply:   0%|          | 0/1678 [00:00<?, ?it/s]

In [ ]:

revisions_df = pd.DataFrame(revisions.tolist(), columns=["title", 'date', "same_day_views", "prev_day_views", "week_before_views", "month_before_views", 
          "year_before_views", "week_after_views", "day_after_revs", "day_after_2_revs"])

In [ ]:
revisions_df.to_csv('german_events_revisions_2022_2.csv') 
files.download('german_events_revisions_2022_2.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df

,Unnamed: 0,date,link,title,text
0,0,2022-01-01,/wiki/Berlin,Berlin,Berlin
1,1,2022-01-01,/wiki/Deutschland,Deutschland,Deutschland
2,2,2022-01-01,/wiki/Eintagsk%C3%BCken#Tötung_männlicher_Eint...,Eintagsk%C3%BCken#Tötung_männlicher_Eintagsküken,Eintagsküken
3,3,2022-01-01,/wiki/Flaschenpfand#Pfandsystem_ab_Mai_2005,Flaschenpfand#Pfandsystem_ab_Mai_2005,Flaschenpfand
4,4,2022-01-01,/wiki/Getr%C3%A4nkedose,Getr%C3%A4nkedose,Getränkedose
...,...,...,...,...,...
3351,3351,2022-12-31,/wiki/Clan_del_Golfo,Clan_del_Golfo,Clan del Golfo
3352,3352,2022-12-31,#cite_note-23,None,NaN
3353,3353,2022-12-31,/wiki/Vatikanstadt,Vatikanstadt,Vatikanstadt
3354,3354,2022-12-31,/wiki/Benedikt_XVI.,Benedikt_XVI.,Benedikt XVI.
